#### Tuesday, December 19, 2023

[Advanced RAG 05 - HyDE - Hypothetical Document Embeddings](https://www.youtube.com/watch?v=v_BnBEubv58)

https://colab.research.google.com/drive/1ncrpizJUITFaZ2E7P3ZV9mpNOOds7TMU?usp=sharing

This uses OpenAI.

This all runs.

In [ ]:
!pip -q install langchain huggingface_hub openai chromadb tiktoken faiss-cpu
!pip -q install sentence_transformers
!pip -q install -U FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.1/448.1 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# !mkdir -p blog_posts
# !unzip -q /content/langchain_blog_posts.zip -d blog_posts

In [1]:
import os
from getpass import getpass

# enter your api key
OPENAI_API_KEY = getpass("Enter your API key: ")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Hypothetical Document Embeddings (HyDE)

modified from - https://github.com/langchain-ai/langchain/tree/master/cookbook

HyDE creates a "Hypothetical" answer with the LLM and then embeds that for search

HyDE = Base Embedding model+ LLM Chain (with prompts)

In [2]:
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import LLMChain, HypotheticalDocumentEmbedder
from langchain.prompts import PromptTemplate

from langchain.document_loaders import TextLoader
import langchain

## BGE Embeddings

In [3]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

In [4]:
# Set up the LLM
llm = OpenAI()

In [5]:
# Load with `web_search` prompt
embeddings = HypotheticalDocumentEmbedder.from_llm(llm,
                                                   bge_embeddings,
                                                   prompt_key="web_search"
                                                   )

In [6]:
embeddings.llm_chain.prompt

PromptTemplate(input_variables=['QUESTION'], template='Please write a passage to answer the question \nQuestion: {QUESTION}\nPassage:')

In [7]:
langchain.debug = True

In [8]:
# Now we can use it as any embedding class!
result = embeddings.embed_query("What items does McDonalds make?")

[llm/start] [1:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Please write a passage to answer the question \nQuestion: What items does McDonalds make?\nPassage:"
  ]
}
[llm/end] [1:llm:OpenAI] [2.56s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " McDonalds is a fast-food restaurant chain that offers a wide variety of menu items. Some of the most popular items they make include hamburgers, cheeseburgers, Big Macs, Chicken McNuggets, French fries, and milkshakes. They also sell salads, wraps, and breakfast items such as Egg McMuffins and sausage burritos. Additionally, they offer desserts like ice cream sundaes, apple pies, and McFlurries. McDonalds also serves a variety of beverages, including coffee, soda, and juice. In addition to their regular menu items, McDonalds also has a selection of seasonal and limited-time items.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        

In [9]:
result

[-0.043985381722450256,
 -0.02784155309200287,
 0.015233228914439678,
 -0.01244358066469431,
 0.06386620551347733,
 0.007529225200414658,
 0.05621625483036041,
 -0.03175361081957817,
 0.049321942031383514,
 0.0002607491333037615,
 -0.006192592903971672,
 -0.046448059380054474,
 -0.022451726719737053,
 0.008747943677008152,
 0.07646042108535767,
 -0.04967319592833519,
 0.02254214696586132,
 -0.08323614299297333,
 -0.0348464734852314,
 -0.04698748141527176,
 0.018519433215260506,
 -0.01463831216096878,
 -0.11095774173736572,
 0.0023250251542776823,
 0.041407328099012375,
 -0.015518611297011375,
 0.04472744092345238,
 -0.0016615722561255097,
 -0.057111941277980804,
 -0.07962136715650558,
 0.04757476970553398,
 -0.06759646534919739,
 0.0413631908595562,
 -0.08403278887271881,
 -0.030919475480914116,
 0.007249536458402872,
 0.025982100516557693,
 -0.039845410734415054,
 0.027974968776106834,
 -0.008053398691117764,
 0.05473636835813522,
 -0.0022630183957517147,
 0.026325928047299385,
 -0.02

## Multiple generations
We can also generate multiple documents and then combine the embeddings for those. By default, we combine those by taking the average. We can do this by changing the LLM we use to generate documents to return multiple things.

In [10]:
multi_llm = OpenAI(n=4, best_of=4)

In [11]:
embeddings = HypotheticalDocumentEmbedder.from_llm(
    multi_llm, bge_embeddings, "web_search"
)

In [12]:
result = embeddings.embed_query("What is McDonalds best selling item?")

[llm/start] [1:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Please write a passage to answer the question \nQuestion: What is McDonalds best selling item?\nPassage:"
  ]
}
[llm/end] [1:llm:OpenAI] [2.97s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " McDonalds is one of the most well-known fast-food restaurants in the world. Its menu is full of classic favorites that have been around for decades. The most popular item on the menu is the Big Mac. The Big Mac consists of two 100% beef patties, lettuce, cheese, pickles, onions, and a special sauce all nestled between a classic sesame seed bun. It is a classic favorite that customers come back for again and again. The Big Mac is McDonalds most popular and best selling item.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      },
      {
        "text": " McDonalds is one of the most popular fast food r

## Using our own prompts
Besides using preconfigured prompts, we can also easily construct our own prompts and use those in the LLMChain that is generating the documents. This can be useful if we know the domain our queries will be in, as we can condition the prompt to generate text more similar to that.

In the example below, let's condition it to generate text about a state of the union address (because we will use that in the next example).

In [13]:
prompt_template = """Please answer the user's question as a single food item
Question: {question}
Answer:"""

prompt = PromptTemplate(input_variables=["question"], template=prompt_template)

llm_chain = LLMChain(llm=llm, prompt=prompt)

In [14]:
embeddings = HypotheticalDocumentEmbedder(
    llm_chain=llm_chain,
    base_embeddings=bge_embeddings
)

In [15]:
result = embeddings.embed_query(
    "What is is McDonalds best selling item?"
)

[llm/start] [1:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Please answer the user's question as a single food item\nQuestion: What is is McDonalds best selling item?\nAnswer:"
  ]
}
[llm/end] [1:llm:OpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Big Mac",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 26,
      "total_tokens": 28,
      "completion_tokens": 2
    },
    "model_name": "text-davinci-003"
  },
  "run": null
}


In [16]:
result

[-0.04409582540392876,
 -0.06810586154460907,
 0.004881984554231167,
 -0.07507439702749252,
 0.061256855726242065,
 -0.00785148423165083,
 0.018615733832120895,
 -0.03252463415265083,
 -0.006711806636303663,
 -0.03244765102863312,
 0.0023166611790657043,
 -0.01737019792199135,
 0.0515742264688015,
 -0.018291117623448372,
 0.05330302193760872,
 0.04405609890818596,
 0.08549802005290985,
 -0.05806003510951996,
 -0.04825190454721451,
 -0.008680153638124466,
 0.03713997080922127,
 -0.047634974122047424,
 -0.05635959282517433,
 0.0047253998927772045,
 -0.03056843765079975,
 -0.0012980042956769466,
 -0.012270244769752026,
 -0.0032627813052386045,
 -0.05424449220299721,
 -0.17446546256542206,
 -0.013889017514884472,
 -0.01773385889828205,
 0.09076354652643204,
 -0.018750648945569992,
 0.025643037632107735,
 0.011270502582192421,
 0.02074555680155754,
 -0.007780749816447496,
 -0.027451669797301292,
 -0.01441165991127491,
 0.09279397130012512,
 0.032126057893037796,
 0.0019034488359466195,
 -0.

## Using HyDE

Now that we have HyDE, we can use it as we would any other embedding class! Here is using it to find similar passages in the state of the union example.

In [18]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

# with open("../../state_of_the_union.txt") as f:
#     state_of_the_union = f.read()

loaders = [
    TextLoader('blogposts/blog.langchain.dev_announcing-langsmith_.txt'),
    TextLoader('blogposts/blog.langchain.dev_benchmarking-question-answering-over-csv-data_.txt'),
    TextLoader('blogposts/blog.langchain.dev_chat-loaders-finetune-a-chatmodel-in-your-voice_.txt'),
]
docs = []
for l in loaders:
    docs.extend(l.load())

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

texts = text_splitter.split_documents(docs) #split_text

In [19]:
texts

[Document(page_content='URL: https://blog.langchain.dev/announcing-langsmith/\nTitle: Announcing LangSmith, a unified platform for debugging, testing, evaluating, and monitoring your LLM applications\n\nLangChain exists to make it as easy as possible to develop LLM-powered applications.\n\nWe started with an open-source Python package when the main blocker for building LLM-powered applications was getting a simple prototype working. We remember seeing Nat Friedman tweet in late 2022 that there was “not enough tinkering happening.” The LangChain open-source packages are aimed at addressing this and we see lots of tinkering happening now (Nat agrees)–people are building everything from chatbots over internal company documents to an AI dungeon master for a Dungeons and Dragons game.', metadata={'source': 'blogposts/blog.langchain.dev_announcing-langsmith_.txt'}),
 Document(page_content='The blocker has now changed. While it’s easy to build a prototype of an application in ~5 lines of Lang

In [20]:
prompt_template = """Please answer the user's question as related to Large Language Models
Question: {question}
Answer:"""

prompt = PromptTemplate(input_variables=["question"], template=prompt_template)

llm_chain = LLMChain(llm=llm, prompt=prompt)

In [21]:
embeddings = HypotheticalDocumentEmbedder(
    llm_chain=llm_chain,
    base_embeddings=bge_embeddings
)

In [22]:
docsearch = Chroma.from_documents(texts, embeddings)

query = "What are chat loaders?"
docs = docsearch.similarity_search(query)

[llm/start] [1:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Please answer the user's question as related to Large Language Models\nQuestion: What are chat loaders?\nAnswer:"
  ]
}
[llm/end] [1:llm:OpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Chat loaders are language models specifically designed to generate natural language responses to user queries in a conversational context.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 24,
      "total_tokens": 46,
      "completion_tokens": 22
    },
    "model_name": "text-davinci-003"
  },
  "run": null
}


In [23]:
print(docs[0].page_content)

URL: https://blog.langchain.dev/chat-loaders-finetune-a-chatmodel-in-your-voice/
Title: Chat Loaders: Fine-tune a ChatModel in your Voice

Summary

We are adding a new integration type, ChatLoaders, to make it easier to fine-tune models on your own unique writing style. These utilities help convert data from popular messaging platforms to chat messages compatible with fine-tuning formats like that supported by OpenAI.

Thank you to Greg Kamradt for Misbah Syed for their thought leadership on this.

Important Links:

Context

On Tuesday, OpenAI announced improved fine-tuning support, extending the service to larger chat models like GPT-3.5-turbo. This enables anyone to customize these larger, more capable models for their own use cases. They also teased support for fine-tuning GPT-4 later this year.

While fine-tuning is typically not advised for teaching an LLM substantially new knowledge or for factual recall; it is good for style transfer.
